In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('../input/credit-card-customers/BankChurners.csv')
pd.set_option('display.max_columns', None)
data.head()

In [ ]:
data = data.drop(data.iloc[:, 21:], 
                       axis = 1) 
data.head()

In [ ]:
data.info()

In [ ]:
data.isna().sum()

In [ ]:
for i in data.columns:
    if 'object' in str(data[str(i)].dtype):
        print(data[str(i)].unique())

In [ ]:
data = pd.get_dummies(data, drop_first=True)
data.head()

In [ ]:
print(data.shape)

In [ ]:
data.info()

we will try that: label encode of income and check final accuracy

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(18,18))
sns.heatmap(data.corr())

In [ ]:
#sns.pairplot(data)

In [ ]:
data.describe()
data.columns

In [ ]:
from numpy import asarray
from sklearn.preprocessing import MinMaxScaler
# define min max scaler
norm = MinMaxScaler().fit(data)
data_norm_arr = norm.transform(data)
data_norm=pd.DataFrame(data=data_norm_arr, columns = ['CLIENTNUM', 'Customer_Age', 'Dependent_count', 'Months_on_book',
       'Total_Relationship_Count', 'Months_Inactive_12_mon',
       'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal',
       'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt',
       'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio',
       'Attrition_Flag_Existing Customer', 'Gender_M',
       'Education_Level_Doctorate', 'Education_Level_Graduate',
       'Education_Level_High School', 'Education_Level_Post-Graduate',
       'Education_Level_Uneducated', 'Education_Level_Unknown',
       'Marital_Status_Married', 'Marital_Status_Single',
       'Marital_Status_Unknown', 'Income_Category_$40K - $60K',
       'Income_Category_$60K - $80K', 'Income_Category_$80K - $120K',
       'Income_Category_Less than $40K', 'Income_Category_Unknown',
       'Card_Category_Gold', 'Card_Category_Platinum', 'Card_Category_Silver'])

In [ ]:
data_norm.describe()

In [ ]:
data_norm.shape

In [ ]:
data_norm.head()

In [ ]:
y= data_norm["Attrition_Flag_Existing Customer"]
y.unique()
x=data_norm.drop("Attrition_Flag_Existing Customer", axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

In [ ]:
x_test

In [ ]:
import xgboost as xgb
classifier=xgb.XGBClassifier()
classifier.fit(x_train,y_train)

In [ ]:
print("Coefficient of determination R^2 <-- on train set: {}".format(classifier.score(x_train, y_train)))

In [ ]:
from sklearn.model_selection import cross_val_score
score=cross_val_score(classifier,x,y,cv=5)

In [ ]:
score.mean()

In [ ]:
xgb_pred=classifier.predict(x_test)

In [ ]:
xgb_pred

In [ ]:
y_test

In [ ]:
import matplotlib.pyplot as plt
data.plot(x = "Gender_M", y = "Attrition_Flag_Existing Customer", kind = "hist")
plt.show()

Less females are existing customers than males

In [ ]:
import matplotlib.pyplot as plt
data.plot(x = "Attrition_Flag_Existing Customer", y = "Customer_Age", kind = "hist")
plt.show()

In [ ]:
df1= data[["Gender_M","Customer_Age", "Attrition_Flag_Existing Customer"]]
df1.groupby(["Gender_M","Customer_Age"]).sum()

In [ ]:
orig_data = pd.read_csv('../input/credit-card-customers/BankChurners.csv')
pd.set_option('display.max_columns', None)
orig_data.head()
orig_data = orig_data.drop(orig_data.iloc[:, 21:], 
                       axis = 1) 
orig_data.head()
orig_data.groupby(["Gender","Customer_Age"]).sum()

In [ ]:
#example 1
pd.set_option('display.max_rows', None)
result = data[['Gender_M','Attrition_Flag_Existing Customer','Customer_Age']].groupby(['Gender_M','Customer_Age']).agg(['count','sum']).reset_index()
result

In [ ]:
result['Prob_churn'] = 1-(result.iloc[:,3]/result.iloc[:,2])
result

In [ ]:
result.sort_values(by='Prob_churn', ascending=False, inplace=True)
result

In [ ]:
result.groupby(by=['Gender_M', 'Customer_Age']).sum().sort_values(by='Prob_churn', ascending=False)

In [ ]:
plt.hist(result['Prob_churn'])#, bins=int(150/5))

In [ ]:
lst= result.columns
lst

In [ ]:
sns.distplot(result[('Attrition_Flag_Existing Customer', 'sum')], hist=True, kde=True, 
             bins=int(180/5), color = 'darkblue', 
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4})

In [ ]:
sns.distplot(result[('Prob_churn', '')], hist=True, kde=True, 
             bins=int(180/5), color = 'darkblue', 
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4})

In [ ]:
result['Churn']=result.iloc[:,1]-result.iloc[:,2]
sns.distplot(result[('Churn', '')], hist=True, kde=True, 
             bins=int(180/5), color = 'darkblue', 
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4})

In [ ]:
sns.distplot(result[('Attrition_Flag_Existing Customer', 'count')], hist=True, kde=True, 
             bins=int(180/5), color = 'darkblue', 
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4})

In [ ]:
data[['Gender_M','Attrition_Flag_Existing Customer','Customer_Age']].groupby(['Gender_M','Customer_Age']).sum().sort_values(by='Attrition_Flag_Existing Customer', ascending=True)